In [1]:
import os
import re
import json
import pandas as pd
from src.utils import read_config
from prompts import gemini_prompts
import google.generativeai as genai
import google.ai.generativelanguage as glm

from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS, DistanceStrategy
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

Building prefix dict from d:\github\project_envelope\backend\dict\dict.big.txt ...
Loading model from cache C:\Users\Sean\AppData\Local\Temp\jieba.u72d3e31b19648520dde69c2a4ddf48a5.cache
Loading model cost 1.067 seconds.
Prefix dict has been built successfully.
e:\miniconda3\envs\project_envelop\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [45]:
organized_result = pd.read_csv('temp_result/organized_result_2.csv')
organized_result.head()

,source,target,reason,fact,keywords
0,34829.pdf,行政院原住民族委員會及花蓮縣政府,花蓮區漁會辦理「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機採購案」，未能確實...,據立法委員函轉民眾陳訴行政院原住民族委員會(下稱原民會)補助花蓮區漁會給二予漁民架設衛星導航...,"['政府', '行政院', '案件', '原住民', '計畫', '漁探機', '本院', ..."
1,34838.pdf,行政院衛生署,為行政院衛生署辦理九十至九十三年度補、捐助計畫，經該署檢討至少計一二五項計畫應以委辦而竟以補...,本案係立法院鄭委員三元向本院陳訴略以「行政院衛生署將應以委辦執行之經費，改由補助費方式辦理，...,"['計畫', '捐助', '年度', '行政院', '案件', '意旨', '下列', '方..."
2,36946.pdf,台北縣萬里鄉公所、台北市政府、基隆市政府、台北縣政府,台北市政府、基隆市政府及台北縣政府未善盡土地使用管制、環境污染稽查及申報資料勾稽、查核之責，...,本案緣陳訴人檢附採證照片陳訴台北市政府核准設立之新生代環境清潔公司（下二稱新生代公司）、統徠...,"['政府', '公司', '清除', '垃圾', '顯有', '核准', '必要性', '欠..."
3,36997.pdf,行政院國軍退除役官兵輔導委員會台北榮民總醫院,行政院國軍退除役官兵輔導委員會台北榮民總醫院辦理「醫學科技大樓法定停車場工程」興建發包作業，...,本案前經審計部函報行政院國軍退除役官兵輔導委員會（以下簡稱退輔會）台北榮民總醫院（以下簡稱台...,"['工程', '提升', '於九', '法定', '完成', '台北', '法令', '方式..."
4,37008.pdf,行政院國軍退除役官兵輔導委員會,為行政院國軍退除役官兵輔導委員會為改善所屬十一所榮民醫院及十八所安養機構之污水處理廠及衛生下...,行政院國軍退除役官兵輔導委員會﹙以下簡稱退輔會﹚為改善所屬十一所榮民醫院及十八所安養機構之廢...,"['工程', '政府', '下水道', '方式', '核有', '污水', '蒐集', '曲..."


In [3]:
configs = read_config('.env/configs.json')
api_key =configs['g_key']
os.environ["GOOGLE_API_KEY"] = api_key
genai.configure(api_key=api_key)

In [4]:
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    convert_system_message_to_human=True,
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
)

In [5]:
kg_reseult = organized_result.copy()
for i, row in kg_reseult.iterrows():
    
    triple_prompt = PromptTemplate.from_template(gemini_prompts.TRIPLE_PROMPT)
    chain = triple_prompt | llm
    triple_data = {
        "reference": row['reason']
    }

    triple_response = chain.invoke(triple_data)

    triple_response_result = re.sub(r'[```json\n]', '', triple_response.content).replace("實體", "entity").replace("關係", "relationship").split("、")[0]

    to_json_prompt = PromptTemplate.from_template(gemini_prompts.TO_JSON_PROMPT)
    chain = to_json_prompt | llm
    data = {
        "reference": triple_response_result
    }

    to_json_response = chain.invoke(data)
    
    
    triple_json_result = re.sub(r'[\n]','',to_json_response.content)

    # triple_json_result = json.loads(f'[{triple_json_result}]')
    
    kg_reseult.loc[kg_reseult.index==i, 'kg'] = str(triple_json_result)
    
    

e:\miniconda3\envs\project_envelop\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
e:\miniconda3\envs\project_envelop\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
e:\miniconda3\envs\project_envelop\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
e:\miniconda3\envs\project_envelop\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
e:\miniconda3\envs\project_envelop\lib\site-packages\langchain_google_ge

In [74]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 500,
    length_function= len
)

In [102]:
kg_reseult = kg_reseult.rename(columns={'ids':'source'})

In [103]:
reason_loader = DataFrameLoader(kg_reseult, page_content_column='reason')
reason_data = reason_loader.load()

# target_loader = DataFrameLoader(organized_result, page_content_column='target')
# target_data = target_loader.load()

reason_documents = text_splitter.transform_documents(reason_data)
print("已為所有文件進行 chunk ",len(reason_documents),"筆")

已為所有文件進行 chunk  75 筆


In [76]:
reason_documents[0].metadata

{'ids': '34829.pdf',
 'target': '行政院原住民族委員會及花蓮縣政府',
 'fact': '據立法委員函轉民眾陳訴行政院原住民族委員會(下稱原民會)補助花蓮區漁會給二予漁民架設衛星導航系統疑有弊端，惟該會遲未盡查處之責，涉有違失等情經本院調查結果，行政院原住民族委員會及花蓮縣政府辦理補助花蓮區漁會採購「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機」乙案，確有下列違失一、花蓮區漁會辦理「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機採購案」，未能確實依政府採購法之規定辦理，採購作業錯誤叢生，補助機關行政院原住民族委員會及花蓮縣政府原住民行政局顯然疏於監督，核有未當，應予檢討改進(一)查行政院原住民族委員會民國(下同)九十二年度補助花蓮縣政府支付花蓮區漁會辦理部落產業發展計畫｜東海岸原住民漁業輔導計畫經費新台幣(下同)一七七萬元，並納入該府預算執行，其中花蓮區漁會辦理「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機採購案」，於九十二年十月十四日開標，由友漁企業公司以一二五萬元得標，惟經查該漁會辦理該採購案違反政府採購法諸多規定，核有下列缺失１、該漁會收受三家廠商投標文件，未於標封袋登記廠商送達方式與時間，亦無簽收紀錄可稽；審查廠商投標作業，僅於證件封袋上予以勾選，無審查之人員簽名負責；又未能提供廠商繳納押標金之紀錄憑核，對於其繳交之額度、支票號碼及領回日期等資料，均無從查核２、該採購案並非特殊或鉅額採購，卻限定投標資格為資本額新台幣五百萬元以上之廠商，上開廠商須具有相當財力之規定，核與政府採購法第三十六條第二項暨「投三標廠商資格與特殊或巨額採購認定標準」第五條第一項第三款規定不符核有非屬特殊或鉅額採購，卻限定財力之特定資格，不當限制廠商競爭之情事，顯已違反採購法第三十七條「機關訂定投標廠商之資格，不得不當限制競爭，並以確認廠商具備履行契約所必須之能力者為限」之規定３、該採購案招標公告刊登標的名稱為「漁船衛星定位儀及通訊系統工程採購」及標的分類為「漁業工程」，惟查採購合約名稱為「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機」，且實際辦理內容則為設備類之財物採購，核有採購屬性刊登錯誤４、該採購案未得標廠商陸青科技股份有限公司，其營利事業登記證、經濟部公司執照及交通部電信管制器材經營許可執照，

In [14]:
model_name = "sentence-transformers/distiluse-base-multilingual-cased-v1"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

In [10]:
# vector_db_path = "./vector_db_2/"

# if not os.path.exists(vector_db_path):
#     os.makedirs(vector_db_path)
#     print(f"Created folder: {vector_db_path}")
    
# vector_store = FAISS.from_documents(reason_documents, embeddings, normalize_L2=True, distance_strategy=DistanceStrategy.MAX_INNER_PRODUCT)
# vector_store.save_local(folder_path=vector_db_path)

In [104]:
id_list = kg_reseult['source'].to_list()

In [106]:
vector_db_path = "./chroma_db/"
chroma_collection_name = "corretion_v1"
if not os.path.exists(vector_db_path):
    os.makedirs(vector_db_path)
    print(f"Created folder: {vector_db_path}")
    
vector_store = Chroma.from_documents(documents=reason_documents, 
                                     persist_directory=vector_db_path, 
                                     collection_name=chroma_collection_name, 
                                     ids=id_list,
                                     embedding=embeddings)

In [110]:
vectorstore = Chroma(persist_directory=vector_db_path, collection_name=chroma_collection_name, embedding_function=embeddings)
vectorstore._collection.count()

75

In [105]:
# vector_store.delete_collection()